In [1]:
import tkinter as tk
from tkinter import messagebox, ttk
from datetime import datetime, timedelta
import requests
import folium
import webbrowser
import os
import xgboost as xgb  # Thêm dòng này để import thư viện xgboost
import numpy as np
model = xgb.XGBRegressor()
model.load_model("xgb_model.json")

input_data = {}

weather_conditions_options = ['Sunny', 'Cloudy', 'Stormy', 'Fog', 'Windy', 'Sandstorms']
road_traffic_density_options = ['Low', 'Medium', 'Jam', 'High']
festival_options = ["No", "Yes"]
city_options = ['Metropolitan', 'Urban', 'Semi-Urban']
vehicle_options = ["Electric Scooter", "Motorcycle", "Scooter"]
order_options = ["Buffet", "Drinks", "Meal", "Snack"]

def calculate_distance(lat1, lon1, lat2, lon2):
    url = f"http://router.project-osrm.org/route/v1/driving/{lon1},{lat1};{lon2},{lat2}?overview=false"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data['routes'][0]['distance'] / 1000  # Convert to kilometers
    else:
        raise Exception("Error calculating distance from OSRM API")

def update_distance_and_predict(*args):
    # Lấy giá trị từ các ô nhập liệu
    restaurant_lat_str = entry_restaurant_lat.get().strip()
    restaurant_lon_str = entry_restaurant_lon.get().strip()
    delivery_lat_str = entry_delivery_lat.get().strip()
    delivery_lon_str = entry_delivery_lon.get().strip()
    
    # Kiểm tra xem các giá trị có thể chuyển đổi sang float không
    if (restaurant_lat_str and restaurant_lon_str and
        delivery_lat_str and delivery_lon_str):
        try:
            restaurant_lat = float(restaurant_lat_str)
            restaurant_lon = float(restaurant_lon_str)
            delivery_lat = float(delivery_lat_str)
            delivery_lon = float(delivery_lon_str)
            
            # Tính khoảng cách
            distance = calculate_distance(restaurant_lat, restaurant_lon, delivery_lat, delivery_lon)
            
            # Cập nhật ô nhập liệu khoảng cách
            entry_distance.delete(0, tk.END)
            entry_distance.insert(0, f"{distance:.2f}")  # Hiển thị khoảng cách trong ô nhập liệu
            
            # Gọi hàm dự đoán thời gian giao hàng
            predict_time()
        except ValueError:
            # Nếu không thể chuyển đổi sang float, xóa nội dung của ô nhập liệu khoảng cách
            entry_distance.delete(0, tk.END)
    else:
        # Nếu không đủ giá trị, xóa nội dung của ô nhập liệu khoảng cách
        entry_distance.delete(0, tk.END)

def predict_time():
    try:
        # Lấy DL từ phần nhập vào
        input_data['Delivery_person_Age'] = float(entry_age.get())
        input_data['Delivery_person_Ratings'] = float(entry_ratings.get())
        input_data['Restaurant_latitude'] = float(entry_restaurant_lat.get())
        input_data['Restaurant_longitude'] = float(entry_restaurant_lon.get())
        input_data['Delivery_location_latitude'] = float(entry_delivery_lat.get())
        input_data['Delivery_location_longitude'] = float(entry_delivery_lon.get())
        
        # Sử dụng ordinal 
        input_data['Weather_conditions'] = weather_conditions_options.index(entry_weather.get())
        input_data['Road_traffic_density'] = road_traffic_density_options.index(entry_traffic.get())
        input_data['Festival'] = festival_options.index(entry_festival.get())
        input_data['City'] = city_options.index(entry_city.get())

        input_data['Vehicle_condition'] = float(entry_vehicle_condition.get())
        input_data['multiple_deliveries'] = float(entry_multiple_deliveries.get())
        input_data['distance_km'] = float(entry_distance.get()) if entry_distance.get() else 0.0
        
        # Trích xuất date
        date_str = entry_date.get()
        date = datetime.strptime(date_str, "%d/%m/%Y")
        input_data['day'] = date.day
        input_data['month'] = date.month
        input_data['quarter'] = (date.month - 1) // 3 + 1
        input_data['year'] = date.year
        input_data['day_of_week'] = date.weekday()
        input_data['is_month_start'] = 1 if date.day == 1 else 0
        input_data['is_month_end'] = 1 if date.day == (date.replace(month=date.month+1, day=1) - timedelta(days=1)).day else 0
        input_data['is_quarter_start'] = 1 if date.day == 1 and date.month in [1, 4, 7, 10] else 0
        input_data['is_quarter_end'] = 1 if date.day == (date.replace(month=date.month+1, day=1) - timedelta(days=1)).day and date.month in [3, 6, 9, 12] else 0
        input_data['is_year_start'] = 1 if date.month == 1 and date.day == 1 else 0
        input_data['is_year_end'] = 1 if date.month == 12 and date.day == 31 else 0
        input_data['is_weekend'] = 1 if date.weekday() >= 5 else 0

        input_data['order_prepare_time'] = float(entry_order_prepare_time.get())
        
        # One-hot 
        vehicle_type = vehicle_var.get()
        for vehicle in vehicle_options:
            input_data[f'Type_of_vehicle_{vehicle.lower().replace(" ", "_")}'] = 1 if vehicle_type == vehicle else 0

        # One-hot 
        order_type = order_var.get()
        for order in order_options:
            input_data[f'Type_of_order_{order.lower()}'] = 1 if order_type == order else 0

        # Chuyển về dạng 2D
        input_array = np.array([list(input_data.values())])

        
        prediction = model.predict(input_array)
        total_delivery_time = prediction[0] + input_data['order_prepare_time']

        # Hiển thị dự đoán
        messagebox.showinfo("Prediction", f"Dự đoán thời gian giao hàng: {total_delivery_time:.2f} phút")
    except Exception as e:
        messagebox.showerror("Error", str(e))

# Lấy map HTML with Folium
def generate_map_html():
    m = folium.Map(location=[11.003669, 76.976494], zoom_start=16)
    m.add_child(folium.LatLngPopup())

    map_html = f"""
    <html>
    <head>
        <script>
            function sendCoordinates(lat, lng) {{
                alert("Latitude: " + lat + "\\nLongitude: " + lng);
            }}
        </script>
    </head>
    <body>
        {m._repr_html_()}
    </body>
    </html>
    """

    html_path = os.path.join(os.getcwd(), 'map.html')
    with open(html_path, 'w', encoding='utf-8') as f:
        f.write(map_html)

    return html_path

def open_map():
    html_path = generate_map_html()
    webbrowser.open('file://' + html_path)

# Tạo và hiển thị map
def create_map():
    try:
        res_lat = float(entry_restaurant_lat.get())
        res_lon = float(entry_restaurant_lon.get())
        del_lat = float(entry_delivery_lat.get())
        del_lon = float(entry_delivery_lon.get())

        location = (res_lat, res_lon)  # Vị trí nhà hàng
        location2 = (del_lat, del_lon)  # Vị trí giao hàng

        # Tạo bản đồ
        m = folium.Map(location=location, zoom_start=13)
        
        # Thêm điểm đánh dấu
        folium.Marker(location, popup="Nhà hàng").add_to(m)
        folium.Marker(location2, popup="Điểm giao hàng").add_to(m)
        
        # Gọi OSRM để lấy thông tin tuyến đường
        url = f"http://router.project-osrm.org/route/v1/driving/{res_lon},{res_lat};{del_lon},{del_lat}?overview=full&geometries=geojson"
        response = requests.get(url)
        data = response.json()

        # Lấy thông tin tuyến đường
        route = data['routes'][0]['geometry']
        
        # Vẽ tuyến đường lên bản đồ
        folium.GeoJson(route).add_to(m)
        
        # Lưu bản đồ vào file HTML và mở trong trình duyệt
        map_file = "map.html"
        m.save(map_file)
        webbrowser.open('file://' + os.path.realpath(map_file))

    except ValueError: 
        messagebox.showerror("Lỗi đầu vào", "Vui lòng nhập giá trị hợp lệ cho vĩ độ và kinh độ.")
    except Exception as e:
        messagebox.showerror("Lỗi", f"Đã xảy ra lỗi: {str(e)}")

# Reset
def reset_fields():
    for entry in entries:
        if isinstance(entry, ttk.Combobox):
            entry.current(0)
        else:
            entry.delete(0, tk.END)

# Giao diện
root = tk.Tk()
root.title("Delivery Time Prediction")


title_label = tk.Label(root, text="Dự đoán thời gian giao hàng", font=("Arial", 18, "bold"), fg="blue")
title_label.grid(row=0, column=0, columnspan=4, pady=10)


labels = ["Tuổi người giao hàng", "Xếp hạng người giao hàng", "Vĩ độ nhà hàng", "Kinh độ nhà hàng", 
          "Vĩ độ điểm giao", "Kinh độ điểm giao", "Điều kiện thời tiết", "Mật độ giao thông đường bộ", 
          "Tình trạng phương tiện", "Giao hàng nhiều lần", "Lễ hội", "Thành phố", "Ngày giao hàng (dd/mm/yyyy)", 
          "Thời gian chuẩn bị đơn hàng", "Phương tiện", "Loại đơn hàng", "Khoảng cách (km)"]

entries = []

row = 1
column = 0

for label in labels:
    tk.Label(root, text=label).grid(row=row, column=column*2, padx=10, pady=5, sticky='w')
    if label in ["Điều kiện thời tiết", "Mật độ giao thông đường bộ", "Lễ hội", "Thành phố", "Phương tiện", "Loại đơn hàng"]:
        entry = ttk.Combobox(root, state="readonly")
    else:
        entry = tk.Entry(root)
    entry.grid(row=row, column=column*2+1, padx=10, pady=5)
    entries.append(entry)
    row += 1
    if row % 10 == 0:  # Adjust to fit your layout
        row = 1
        column += 1

(entry_age, entry_ratings, entry_restaurant_lat, entry_restaurant_lon,
 entry_delivery_lat, entry_delivery_lon, entry_weather, entry_traffic,
 entry_vehicle_condition, entry_multiple_deliveries, entry_festival, entry_city,
 entry_date, entry_order_prepare_time, entry_vehicle,
 entry_order, entry_distance) = entries

# comboboxes
entry_weather['values'] = weather_conditions_options
entry_weather.current(0)

entry_traffic['values'] = road_traffic_density_options
entry_traffic.current(0)

entry_festival['values'] = festival_options
entry_festival.current(0)

entry_city['values'] = city_options
entry_city.current(0)

entry_vehicle['values'] = vehicle_options
entry_vehicle.current(0)

entry_order['values'] = order_options
entry_order.current(0)

vehicle_var = tk.StringVar(value=vehicle_options[0])
order_var = tk.StringVar(value=order_options[0])

entry_vehicle['textvariable'] = vehicle_var
entry_order['textvariable'] = order_var

entry_restaurant_lat.bind('<FocusOut>', update_distance_and_predict)
entry_restaurant_lon.bind('<FocusOut>', update_distance_and_predict)
entry_delivery_lat.bind('<FocusOut>', update_distance_and_predict)
entry_delivery_lon.bind('<FocusOut>', update_distance_and_predict)

predict_button = tk.Button(root, text="Dự đoán thời gian giao hàng", bg="white", fg="green", font=("Helvetica", 14, "bold"), command=predict_time)

predict_button.grid(row=12, column=0, columnspan=4, pady=20)

select_restaurant_button = tk.Button(root, text="Chọn vị trí trên bản đồ", command=open_map)
select_restaurant_button.grid(row=11, column=3, pady=5)

reset_button = tk.Button(root, text="Reset", command=reset_fields)
reset_button.grid(row=12, column=3, pady=20)

create_map_button = tk.Button(root, text="Bản đồ", command=create_map)
create_map_button.grid(row=13, column=3, pady=20)

root.mainloop()
